In [1]:
import pymortar
import glob
URL = "http://mortar2_mortar-server_1:5001"
# connect client to Mortar frontend server
c = pymortar.Client(URL)

In [2]:
# query AHU, supply air temp sensor, and downstream zone
query1 = """SELECT ?ahu ?sat ?zone WHERE {
    ?ahu a brick:AHU .
    ?ahu brick:hasPart?/brick:hasPoint ?sat .
    ?sat a brick:Supply_Air_Temperature_Sensor .
    ?ahu brick:feeds+ ?zone .
    ?zone a brick:HVAC_Zone
}"""

# damper position commands in VAVs
query2 = """SELECT ?vav ?cmd WHERE {
    ?vav   a  brick:VAV .
    ?vav   brick:hasPart?/brick:hasPoint ?cmd .
    ?cmd a brick:Damper_Position_Command
}"""

# all air temperature sensors
query3 = """SELECT ?ats WHERE {
    ?ats  a  brick:Air_Temperature_Sensor .
}"""

## Querying (Brick) Metadata

In [3]:
# execute query against all sites
# c.sparql(query)

# or against a list of sites
c.sparql(query1, sites=['bldg1'])

,ahu,sat,zone,site
0,http://buildsys.org/ontologies/BLDG1#AHU07A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM091_LAB,bldg1
1,http://buildsys.org/ontologies/BLDG1#AHU07A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM190_LAB,bldg1
2,http://buildsys.org/ontologies/BLDG1#AHU07A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM280F_LAB,bldg1
3,http://buildsys.org/ontologies/BLDG1#AHU07A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM180_LAB,bldg1
4,http://buildsys.org/ontologies/BLDG1#AHU07A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM285_LAB,bldg1
...,...,...,...,...
389,http://buildsys.org/ontologies/BLDG1#AHU04A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM034_LAB,bldg1
390,http://buildsys.org/ontologies/BLDG1#AHU04A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM041_LAB,bldg1
391,http://buildsys.org/ontologies/BLDG1#AHU04A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM246_LAB,bldg1
392,http://buildsys.org/ontologies/BLDG1#AHU04A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM148_LAB,bldg1


In [4]:
# get a summary of which sites return results for a set of queries
c.qualify([query1, query2, query3])

       Query_0  Query_1  Query_2
bldg1      394        0      829
bldg2        0        0        1

In [5]:
# name the queries (optional)
c.qualify({"damper_cmd": query2, "ahu sat": query1, "temp sensors": query3})

       damper_cmd  ahu sat  temp sensors
bldg1           0      394           829
bldg2           0        0             1

## Querying Timeseries Data

In [6]:
# get metadata, data tables as separate objects
md, df = c.get_data_sparql(query3)

In [7]:
# metadata for all streams in the dataframe
md

,ats
0,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
1,http://example.com#mysensortest
2,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
3,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
4,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
...,...
825,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
826,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
827,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
828,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...


In [8]:
# timeseries data for all streams in the metadata table
df

,time,value,id,unit,type
0,2015-01-01 00:00:00+00:00,69.0,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,https://brickschema.org/schema/Brick#Supply_Ai...
1,2015-01-01 00:15:00+00:00,64.0,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,https://brickschema.org/schema/Brick#Supply_Ai...
2,2015-01-01 00:30:00+00:00,69.5,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,https://brickschema.org/schema/Brick#Supply_Ai...
3,2015-01-01 00:45:00+00:00,61.5,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,https://brickschema.org/schema/Brick#Supply_Ai...
4,2015-01-01 01:00:00+00:00,61.0,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,https://brickschema.org/schema/Brick#Supply_Ai...
...,...,...,...,...,...
158396,2015-01-02 23:00:00+00:00,73.5,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,https://brickschema.org/schema/Brick#Zone_Air_...
158397,2015-01-02 23:15:00+00:00,73.5,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,https://brickschema.org/schema/Brick#Zone_Air_...
158398,2015-01-02 23:30:00+00:00,74.0,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,https://brickschema.org/schema/Brick#Zone_Air_...
158399,2015-01-02 23:45:00+00:00,74.0,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,https://brickschema.org/schema/Brick#Zone_Air_...
